In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd


path = "/content/drive/My Drive/fire_predict/base_data/"
df_train = pd.read_csv(path + "PJT002_train.csv")
df_val = pd.read_csv(path + "PJT002_validation.csv")
df_test = pd.read_csv(path + "PJT002_test.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,172,173,174,175,176,177,178,179) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
import math
import numpy as np
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from tqdm import tqdm


def f_data(train, val, test):

    X_train = train.drop(['fr_yn'], 1)
    y_train = train['fr_yn']
    X_val = val.drop(['fr_yn'], 1)
    y_val = val['fr_yn']
    X_test = test.drop(['fr_yn'], 1)

    df_all = pd.concat([X_train, X_val, X_test])

    categorical_cols = df_all.select_dtypes(['object']).columns
    for col in categorical_cols:
        df_all[col] = pd.Categorical(df_all[col]).codes

    X_train = df_all[:len(train)]
    X_val = df_all[len(train):-len(test)]
    X_test = df_all[-len(test):]

    # X_train = X_train.fillna(-1)
    # X_val = X_val.fillna(-1)
    # X_test = X_test.fillna(-1)

    X_train = X_train.replace({-1:np.NaN})
    X_val = X_val.replace({-1:np.NaN})
    X_test = X_test.replace({-1:np.NaN})

    return X_train, y_train, X_val, y_val, X_test

def f1_rfc(X_train, y_train, X_val, y_val):
    result_list = []
    for i in tqdm(range(10)):

        model = RandomForestClassifier(bootstrap=True, max_depth=80,
                                       max_features="auto", min_samples_leaf=4,
                                       min_samples_split=5, n_estimators=200,
                                       random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)
        result = f1_score(y_val, y_pred)
        result_list.append(result)
    return sum(result_list) / len(result_list)

def f1_rfc_2(X_train, y_train, X_val, y_val):
    result_list = []
    for i in tqdm(range(10)):

        model = RandomForestClassifier(random_state=42, n_estimators=1000)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)
        result = f1_score(y_val, y_pred)
        result_list.append(result)
    return sum(result_list) / len(result_list)

# lable -> binary
binary_y = {'N': 0, 'Y': 1}

df_train['fr_yn'] = df_train['fr_yn'].map(binary_y)
df_val['fr_yn'] = df_val['fr_yn'].map(binary_y)

#df ->df_2
df_train_2 = df_train
df_val_2 = df_val
df_test_2 = df_test

#drop id
df_train_2 = df_train_2.drop("id", axis=1)
df_val_2 = df_val_2.drop("id", axis=1)
df_test_2 = df_test_2.drop("id", axis=1)

#drop dt_of_fr
df_train_2 = df_train_2.drop("dt_of_fr", axis=1)
df_val_2 = df_val_2.drop("dt_of_fr", axis=1)
df_test_2 = df_test_2.drop("dt_of_fr", axis=1)

#dt_of_athrztn -> year 
def year(x):
    if x != "NaN":
        if isinstance(x, float):
            if not math.isnan(x):
                if int(x) >10000000:
                    return x // 10000
                elif int(x) > 1000000:
                    return x // 1000
                elif int(x) > 100000:
                    return x // 100
                elif int(x) > 10000:
                    return x // 10
                elif int(x) > 1000:
                    return x
        elif isinstance(x, str):
            return int(x[:4])

df_train_2["dt_of_athrztn"] = df_train_2["dt_of_athrztn"].apply(year)
df_train_2["dt_of_athrztn"] = df_train_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)
df_val_2["dt_of_athrztn"] = df_val_2["dt_of_athrztn"].apply(year)
df_val_2["dt_of_athrztn"] = df_val_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)
df_test_2["dt_of_athrztn"] = df_test_2["dt_of_athrztn"].apply(year)
df_test_2["dt_of_athrztn"] = df_test_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)

# hm_cnt -> categorical
def population(x):
    if x < 1000:
        return "1k"
    elif x < 10000:
        return "10k"
    elif x < 100000:
        return "100k"
    elif x < 1000000:
        return "1000k"

df_train_2["hm_cnt"] = df_train_2["hm_cnt"].apply(population)
df_val_2["hm_cnt"] = df_val_2["hm_cnt"].apply(population)
df_test_2["hm_cnt"] = df_test_2["hm_cnt"].apply(population)

#tmprtr c -> k
df_train_2.tmprtr = df_train_2.tmprtr.apply(lambda x: x + 273.15 if True else x)
df_val_2.tmprtr = df_val_2.tmprtr.apply(lambda x: x + 273.15 if True else x)
df_test_2.tmprtr = df_test_2.tmprtr.apply(lambda x: x + 273.15 if True else x)

#gas_engry_us_201507 x < 0 -> remove
df_train_2.gas_engry_us_201507 = df_train_2.gas_engry_us_201507.apply(lambda x: None if x < 0 else x)

# drop gas, elec
gas_elec_columns = list(df_train_2.iloc[:,16:136].columns)
df_train_2 = df_train_2.drop(gas_elec_columns, axis=1)
df_val_2 = df_val_2.drop(gas_elec_columns, axis=1)
df_test_2 = df_test_2.drop(gas_elec_columns, axis=1)

#for imputation
s = df_train_2.isnull().sum()
c_list = np.where(s >= max(s)*0.5)
df_train_2 = df_train_2.drop(list(s.index[c_list[0]]), axis=1)

df_val_2 = df_val_2.drop(list(s.index[c_list[0]]), axis=1)
df_test_2 = df_test_2.drop(list(s.index[c_list[0]]), axis=1)

X_train, y_train, X_val, y_val, X_test = f_data(df_train_2, df_val_2, df_test_2)

In [5]:
X_train = pd.read_csv(path + "imputed_34f.csv", header=None)
X_val = X_val.fillna(-1)
X_test = X_test.fillna(-1)
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
0,9.000000,8.000000,3.0,69.420,69.4200,0.0,1977.000000,1.000000,0.000000,6.000000,283.85,0.9,200.0,96.0,4.0,13.0,18.0,12.0,7.0,199.0,1.0,4547.0,6.397596e+05,133.0,137.0,0.0,112.0,0.0,0.0,1891.0,29231.0,11322.0,88.0,0.0
1,12.649990,9.058762,3.0,46.290,46.2900,0.0,1984.654245,1.000000,0.000000,5.215133,292.65,0.5,20.0,74.0,4.0,1.0,18.0,12.0,6.0,141.0,2.0,6388.0,1.225810e+05,489.0,85.0,0.0,818.0,0.0,0.0,4533.0,20480.0,3369.0,7727.0,0.0
2,1.000000,15.000000,1.0,583.800,2516.7600,1446.0,2000.000000,5.000000,0.000000,6.000000,288.65,2.0,90.0,21.0,3.0,17.0,18.0,25.0,11.0,199.0,1.0,3340.0,6.181050e+05,143.0,137.0,0.0,165.0,0.0,0.0,277.0,31197.0,12451.0,72.0,14.0
3,9.000000,10.000000,2.0,48.920,48.9200,0.0,1936.000000,1.000000,0.000000,6.000000,293.75,0.4,0.0,91.0,4.0,15.0,18.0,12.0,7.0,261.0,2.0,179.0,7.195420e+05,1585.0,176.0,0.0,131.0,0.0,0.0,438.0,4270.0,407.0,508.0,11.0
4,14.538810,6.531796,2.0,0.000,0.0000,0.0,1968.814380,1.000000,0.000000,5.692288,285.85,0.6,360.0,89.0,13.0,14.0,18.0,31.0,7.0,199.0,1.0,4822.0,6.397596e+05,603.0,137.0,0.0,686.0,0.0,0.0,1702.0,29778.0,12487.0,707.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59194,10.000000,1.000000,2.0,423.000,423.0000,3010.0,1996.000000,3.121446,0.083291,5.215133,284.35,0.2,0.0,41.0,6.0,1.0,18.0,35.0,6.0,331.0,1.0,3435.0,6.397596e+05,895.0,139.0,0.0,1131.0,0.0,0.0,3591.0,3624.0,6679.0,914.0,0.0
59195,12.649990,9.058762,35.0,5593.711,54835.0382,0.0,1984.654245,171.000000,0.000000,5.215133,287.95,1.9,20.0,72.0,4.0,1.0,18.0,22.0,9.0,331.0,1.0,1137.0,1.849455e+06,88.0,139.0,0.0,108.0,0.0,0.0,2629.0,77.0,1285.0,131.0,0.0
59196,5.407946,10.378813,2.0,2436.820,2468.8000,6512.0,2002.829298,4.000000,0.000000,2.348822,286.45,3.2,230.0,58.0,15.0,1.0,18.0,6.0,11.0,326.0,2.0,5875.0,6.397596e+05,25.0,139.0,0.0,968.0,2.0,0.0,4271.0,6139.0,4878.0,790.0,0.0
59197,27.000000,1.000000,2.0,283.200,207.7100,0.0,2005.000000,2.000000,0.000000,5.000000,291.35,1.4,360.0,95.0,3.0,4.0,18.0,14.0,3.0,325.0,2.0,8579.0,6.397596e+05,510.0,139.0,0.0,177.0,0.0,0.0,1387.0,10142.0,3572.0,613.0,0.0


In [7]:
X_val.columns

Index(['bldng_us', 'bldng_archtctr', 'bldng_cnt', 'bldng_ar', 'ttl_ar',
       'lnd_ar', 'dt_of_athrztn', 'ttl_grnd_flr', 'ttl_dwn_flr',
       'bldng_us_clssfctn', 'tmprtr', 'wnd_spd', 'wnd_drctn', 'hmdt', 'jmk',
       'rgnl_ar_nm', 'rgnl_ar_nm2', 'lnd_us_sttn_nm', 'rd_sd_nm', 'emd_nm',
       'hm_cnt', 'fr_sttn_dstnc', 'bldng_ar_prc', 'fr_wthr_fclt_dstnc',
       'fr_mn_cnt', 'mlt_us_yn', 'cctv_dstnc', 'fr_wthr_fclt_in_100m',
       'cctv_in_100m', 'tbc_rtl_str_dstnc', 'sft_emrgnc_bll_dstnc',
       'ahsm_dstnc', 'no_tbc_zn_dstnc', 'bldng_cnt_in_50m'],
      dtype='object')

In [0]:
X_train.columns = X_val.columns

In [50]:
model = RandomForestClassifier(bootstrap=True, max_depth=80,
                                       max_features="auto", min_samples_leaf=4,
                                       min_samples_split=5, n_estimators=2000,
                                       random_state=42)
model.fit(X_train, y_train)


KeyboardInterrupt: ignored

In [0]:
y_pred = model.predict(X_val)
y_pred

In [0]:
result = f1_score(y_val, y_pred)
print(result)

In [51]:
model = RandomForestClassifier(bootstrap=True, max_depth=80,
                                       max_features="auto", min_samples_leaf=4,
                                       min_samples_split=5, n_estimators=200,
                                       random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=80, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [52]:
y_pred = model.predict(X_val)
result = f1_score(y_val, y_pred)
print(result)

0.4670807453416149


In [0]:
m_f_i = model.feature_importances_

In [0]:
dic_feature = dict()
for i, x in enumerate(X_val.columns):
    dic_feature[x] = m_f_i[i]

In [55]:
dic_feature

{'ahsm_dstnc': 0.03662874885121335,
 'bldng_ar': 0.06758689916480358,
 'bldng_ar_prc': 0.02394060420336089,
 'bldng_archtctr': 0.055526635059012945,
 'bldng_cnt': 0.015693857701067834,
 'bldng_cnt_in_50m': 0.012059379849095046,
 'bldng_us': 0.0767115552672639,
 'bldng_us_clssfctn': 0.03924791535187698,
 'cctv_dstnc': 0.03085593298476356,
 'cctv_in_100m': 0.004144792990070613,
 'dt_of_athrztn': 0.0517754115502013,
 'emd_nm': 0.0325546427572815,
 'fr_mn_cnt': 0.022096396162281595,
 'fr_sttn_dstnc': 0.03425154785563725,
 'fr_wthr_fclt_dstnc': 0.057623542577781274,
 'fr_wthr_fclt_in_100m': 0.005185549074481046,
 'hm_cnt': 0.0034980289596198582,
 'hmdt': 0.05595460511301293,
 'jmk': 0.004745206073351444,
 'lnd_ar': 0.03342113658672665,
 'lnd_us_sttn_nm': 0.013053493245300501,
 'mlt_us_yn': 0.0004560293817270528,
 'no_tbc_zn_dstnc': 0.04222463578723979,
 'rd_sd_nm': 0.013400170380767526,
 'rgnl_ar_nm': 0.011643816673836564,
 'rgnl_ar_nm2': 0.0005777826508116545,
 'sft_emrgnc_bll_dstnc': 0.03

In [0]:
from operator import itemgetter
sorted_dict_feature = sorted(dic_feature.items(), key=itemgetter(1), reverse=True)

In [0]:
sorted_dict_feature[0]

('ttl_ar', 0.06391477292283695)

In [0]:
feature_list = []
max_f1 = 0
for d in sorted_dict_feature:
    print("start", d[0])
    feature_list.append(d[0])
    X_train_2 = X_train[feature_list]
    X_val_2 = X_val[feature_list]
    temp_f1 = f1_rfc(X_train_2, y_train, X_val_2, y_val)
    if temp_f1 < max_f1:
        print("remove", d[0])
        feature_list.remove(d[0])
        continue
    max_f1 = temp_f1
print(feature_list)

  0%|          | 0/10 [00:00<?, ?it/s]

start ttl_ar


  0%|          | 0/10 [00:00<?, ?it/s]

start bldng_ar


  0%|          | 0/10 [00:00<?, ?it/s]

...

remove hm_cnt
start rgnl_ar_nm2


  0%|          | 0/10 [00:00<?, ?it/s]

remove rgnl_ar_nm2
start mlt_us_yn


100%|██████████| 10/10 [04:08<00:00, 24.82s/it]

remove mlt_us_yn
['ttl_ar', 'bldng_ar', 'bldng_us', 'hmdt', 'fr_wthr_fclt_dstnc', 'dt_of_athrztn', 'bldng_archtctr', 'no_tbc_zn_dstnc']


In [0]:
sorted_dict_feature = sorted(dic_feature.items(), key=itemgetter(1))

In [0]:
sorted_dict_feature[0]

('mlt_us_yn', 0.0004560293817270528)

In [0]:
feature_list = [x[0] for x in sorted_dict_feature]
min_f1 = 0.467
for d in sorted_dict_feature:
    print("start", d[0])
    feature_list.remove(d[0])
    X_train_2 = X_train[feature_list]
    X_val_2 = X_val[feature_list]
    temp_f1 = f1_rfc(X_train_2, y_train, X_val_2, y_val)
    if temp_f1 > min_f1:
        print("remove", d[0])
        continue
    feature_list.append(d[0])
    min_f1 = temp_f1
print(feature_list)




  0%|          | 0/10 [00:00<?, ?it/s]

start mlt_us_yn





 10%|█         | 1/10 [00:33<05:02, 33.60s/it]


 20%|██        | 2/10 [01:07<04:28, 33.58s/it]


 30%|███       | 3/10 [01:40<03:54, 33.55s/it]


 40%|████      | 4/10 [02:14<03:21, 33.51s/it]


 50%|█████     | 5/10 [02:47<02:47, 33.50s/it]


 60%|██████    | 6/10 [03:20<02:13, 33.48s/it]


 70%|███████   | 7/10 [03:54<01:40, 33.39s/it]


 80%|████████  | 8/10 [04:27<01:06, 33.29s/it]


 90%|█████████ | 9/10 [05:00<00:33, 33.21s/it]


100%|██████████| 10/10 [05:33<00:00, 33.15s/it]





  0%|          | 0/10 [00:00<?, ?it/s]

start rgnl_ar_nm2





 10%|█         | 1/10 [00:33<04:59, 33.33s/it]


 20%|██        | 2/10 [01:06<04:26, 33.36s/it]


 30%|███       | 3/10 [01:40<03:53, 33.36s/it]


 40%|████      | 4/10 [02:13<03:20, 33.36s/it]


 50%|█████     | 5/10 [02:46<02:46, 33.36s/it]


 60%|██████    | 6/10 [03:20<02:13, 33.35s/it]


 70%|███████   | 7/10 [03:53<01:39, 33.32s/it]


 80%|████████  | 8/10 [04:26<01:06, 33.20s/it]


 90%|█████████ | 9/10 [04:59<00:33, 33.24s/it]


100%|██████████| 10/10 [05:32<00:00, 33.24s/it]





  0%|          | 0/10 [00:00<?, ?it/s]

...

remove bldng_ar
start ttl_ar





 10%|█         | 1/10 [00:18<02:45, 18.35s/it]


 20%|██        | 2/10 [00:36<02:26, 18.35s/it]


 30%|███       | 3/10 [00:55<02:08, 18.35s/it]


 40%|████      | 4/10 [01:13<01:50, 18.34s/it]


 50%|█████     | 5/10 [01:31<01:31, 18.35s/it]


 60%|██████    | 6/10 [01:50<01:13, 18.34s/it]


 70%|███████   | 7/10 [02:08<00:55, 18.34s/it]


 80%|████████  | 8/10 [02:26<00:36, 18.34s/it]


 90%|█████████ | 9/10 [02:45<00:18, 18.34s/it]


100%|██████████| 10/10 [03:03<00:00, 18.36s/it]





  0%|          | 0/10 [00:00<?, ?it/s]

start bldng_us





 10%|█         | 1/10 [00:21<03:10, 21.11s/it]


 20%|██        | 2/10 [00:42<02:48, 21.10s/it]


 30%|███       | 3/10 [01:03<02:27, 21.11s/it]


 40%|████      | 4/10 [01:24<02:06, 21.09s/it]


 50%|█████     | 5/10 [01:45<01:45, 21.08s/it]


 60%|██████    | 6/10 [02:06<01:24, 21.07s/it]


 70%|███████   | 7/10 [02:27<01:03, 21.12s/it]


 80%|████████  | 8/10 [02:48<00:42, 21.10s/it]


 90%|█████████ | 9/10 [03:09<00:21, 21.10s/it]


100%|██████████| 10/10 [03:30<00:00, 21.12s/it]




remove bldng_us
['mlt_us_yn', 'rgnl_ar_nm2', 'hm_cnt', 'ttl_dwn_flr', 'tmprtr', 'lnd_ar', 'sft_emrgnc_bll_dstnc', 'hmdt', 'ttl_ar']


In [57]:
sorted_dict_feature = sorted(dic_feature.items(), key=itemgetter(1), reverse=True)
feature_list = [x[0] for x in sorted_dict_feature]
min_f1 = 0.467
for d in sorted_dict_feature:
    print("start", d[0])
    feature_list.remove(d[0])
    X_train_2 = X_train[feature_list]
    X_val_2 = X_val[feature_list]
    temp_f1 = f1_rfc(X_train_2, y_train, X_val_2, y_val)
    if temp_f1 > min_f1:
        print("remove", d[0])
        continue
    feature_list.append(d[0])
    min_f1 = temp_f1
print(feature_list)




  0%|          | 0/10 [00:00<?, ?it/s]

start bldng_us





 10%|█         | 1/10 [00:33<04:58, 33.16s/it]


 20%|██        | 2/10 [01:06<04:25, 33.18s/it]


 30%|███       | 3/10 [01:39<03:52, 33.18s/it]


 40%|████      | 4/10 [02:12<03:19, 33.19s/it]


 50%|█████     | 5/10 [02:45<02:45, 33.12s/it]


 60%|██████    | 6/10 [03:18<02:12, 33.07s/it]


 70%|███████   | 7/10 [03:51<01:39, 33.14s/it]


 80%|████████  | 8/10 [04:25<01:06, 33.13s/it]


 90%|█████████ | 9/10 [04:58<00:33, 33.20s/it]


100%|██████████| 10/10 [05:31<00:00, 33.13s/it]





  0%|          | 0/10 [00:00<?, ?it/s]

start ttl_ar





 10%|█         | 1/10 [00:32<04:51, 32.39s/it]


 20%|██        | 2/10 [01:04<04:19, 32.40s/it]


 30%|███       | 3/10 [01:37<03:46, 32.41s/it]


 40%|████      | 4/10 [02:09<03:14, 32.38s/it]


 50%|█████     | 5/10 [02:41<02:41, 32.38s/it]


 60%|██████    | 6/10 [03:14<02:09, 32.39s/it]


 70%|███████   | 7/10 [03:46<01:37, 32.38s/it]


 80%|████████  | 8/10 [04:19<01:04, 32.39s/it]


 90%|█████████ | 9/10 [04:51<00:32, 32.43s/it]


100%|██████████| 10/10 [05:23<00:00, 32.41s/it]





  0%|          | 0/10 [00:00<?, ?it/s]

...

remove hm_cnt
start rgnl_ar_nm2





 10%|█         | 1/10 [00:16<02:29, 16.63s/it]


 20%|██        | 2/10 [00:33<02:12, 16.60s/it]


 30%|███       | 3/10 [00:49<01:56, 16.60s/it]


 40%|████      | 4/10 [01:06<01:39, 16.59s/it]


 50%|█████     | 5/10 [01:22<01:22, 16.57s/it]


 60%|██████    | 6/10 [01:39<01:06, 16.58s/it]


 70%|███████   | 7/10 [01:55<00:49, 16.55s/it]


 80%|████████  | 8/10 [02:12<00:33, 16.56s/it]


 90%|█████████ | 9/10 [02:29<00:16, 16.55s/it]


100%|██████████| 10/10 [02:45<00:00, 16.53s/it]





  0%|          | 0/10 [00:00<?, ?it/s]

remove rgnl_ar_nm2
start mlt_us_yn





 10%|█         | 1/10 [00:13<02:04, 13.86s/it]


 20%|██        | 2/10 [00:27<01:50, 13.85s/it]


 30%|███       | 3/10 [00:41<01:36, 13.84s/it]


 40%|████      | 4/10 [00:55<01:23, 13.83s/it]


 50%|█████     | 5/10 [01:09<01:09, 13.83s/it]


 60%|██████    | 6/10 [01:23<00:55, 13.87s/it]


 70%|███████   | 7/10 [01:36<00:41, 13.85s/it]


 80%|████████  | 8/10 [01:50<00:27, 13.84s/it]


 90%|█████████ | 9/10 [02:04<00:13, 13.83s/it]


100%|██████████| 10/10 [02:18<00:00, 13.82s/it]




remove mlt_us_yn
['bldng_us', 'ttl_ar', 'fr_wthr_fclt_dstnc']


In [0]:
l_1 = ['ttl_ar', 'bldng_ar', 'bldng_us', 'hmdt', 'fr_wthr_fclt_dstnc', 'dt_of_athrztn', 'bldng_archtctr', 'no_tbc_zn_dstnc']
l_2 = ['mlt_us_yn', 'rgnl_ar_nm2', 'hm_cnt', 'ttl_dwn_flr', 'tmprtr', 'lnd_ar', 'sft_emrgnc_bll_dstnc', 'hmdt', 'ttl_ar']
l_3 = ['bldng_us', 'ttl_ar', 'fr_wthr_fclt_dstnc']

In [0]:
X_train_2 = X_train[['ttl_ar', 'bldng_ar', 'bldng_us', 'hmdt', 'fr_wthr_fclt_dstnc', 'dt_of_athrztn', 'bldng_archtctr', 'no_tbc_zn_dstnc']]
X_val_2 = X_val[['ttl_ar', 'bldng_ar', 'bldng_us', 'hmdt', 'fr_wthr_fclt_dstnc', 'dt_of_athrztn', 'bldng_archtctr', 'no_tbc_zn_dstnc']]

In [0]:
f1_rfc(X_train, y_train, X_val, y_val)

100%|██████████| 10/10 [05:27<00:00, 32.66s/it]


0.4670807453416149

In [0]:
f1_rfc_2(X_train, y_train, X_val, y_val)

100%|██████████| 10/10 [29:33<00:00, 177.42s/it]


0.4673869547819128

In [0]:
f1_rfc(X_train_2, y_train, X_val_2, y_val)



  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:19<02:55, 19.49s/it]

 20%|██        | 2/10 [00:38<02:35, 19.50s/it]

 30%|███       | 3/10 [00:58<02:16, 19.51s/it]

 40%|████      | 4/10 [01:17<01:56, 19.48s/it]

 50%|█████     | 5/10 [01:37<01:37, 19.49s/it]

 60%|██████    | 6/10 [01:56<01:17, 19.48s/it]

 70%|███████   | 7/10 [02:16<00:58, 19.47s/it]

 80%|████████  | 8/10 [02:35<00:38, 19.46s/it]

 90%|█████████ | 9/10 [02:55<00:19, 19.47s/it]

100%|██████████| 10/10 [03:14<00:00, 19.45s/it]



0.46058252427184454

In [0]:
f1_rfc_2(X_train_2, y_train, X_val_2, y_val)



  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [01:44<15:40, 104.50s/it]

 20%|██        | 2/10 [03:28<13:55, 104.46s/it]

 30%|███       | 3/10 [05:12<12:10, 104.33s/it]

 40%|████      | 4/10 [06:57<10:26, 104.37s/it]

 50%|█████     | 5/10 [08:41<08:41, 104.35s/it]

 60%|██████    | 6/10 [10:26<06:57, 104.35s/it]

 70%|███████   | 7/10 [12:10<05:13, 104.40s/it]

 80%|████████  | 8/10 [13:55<03:28, 104.50s/it]

 90%|█████████ | 9/10 [15:39<01:44, 104.55s/it]

100%|██████████| 10/10 [17:24<00:00, 104.51s/it]



0.45461365341335325

In [0]:
X_train_3 = X_train[l_2]
X_val_3 = X_val[l_2]

In [10]:
f1_rfc(X_train_3, y_train, X_val_3, y_val)

100%|██████████| 10/10 [03:31<00:00, 21.15s/it]


0.406813627254509

In [11]:
f1_rfc_2(X_train_3, y_train, X_val_3, y_val)

100%|██████████| 10/10 [19:42<00:00, 118.01s/it]


0.4155098133078027

In [0]:
X_train_4 = X_train[l_3]
X_val_4 = X_val[l_3]

In [13]:
f1_rfc(X_train_4, y_train, X_val_4, y_val)

100%|██████████| 10/10 [02:18<00:00, 13.83s/it]


0.4369043080442242

In [14]:
f1_rfc_2(X_train_4, y_train, X_val_4, y_val)

100%|██████████| 10/10 [13:01<00:00, 78.07s/it]


0.4049331963001027

In [15]:
X_train_5 = X_train[list(set(X_train_3 + X_train_2 + X_train_4))]
X_val_5 = X_val[list(set(X_train_3 + X_train_2 + X_train_4))]

NameError: ignored

In [0]:
f1_rfc(X_train_5, y_train, X_val_5, y_val)

In [0]:
f1_rfc_2(X_train_5, y_train, X_val_5, y_val)